## INTRODUCTION:

 
   ### A person named Laurel got a job offer from a renowned company and she has to choose from two job locations, Toronto and Newyork City. And she is confused where to move in.  So, this project will analyse the neighborhoods between Toronto, Canada and Newyork City, Newyork. It will give a better overview of these two places describing the similarities and dissimilarities between neighborhoods in the two cities. In this way, Laurel can get insights into the neighborhoods of these cities and will be able to opt the best living standards for her with the job.

### 1. Segmenting, Exploring and Clustering the neighhborhoods of Toronto

### Importing libraries and packages

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
# Obtaining Postal Code, Borough, and Neighborhood information from Wikipedia

table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

toronto_df = table[0]
toronto_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Transforming the Toronto neighborhoods data

In [3]:
toronto_df.rename(columns = {"Postcode": "Postal Code", "Neighbourhood": "Neighborhood"}, inplace = True)

# Dropping 'Not assigned' Borough rows
toronto_df.drop(toronto_df[toronto_df.Borough == 'Not assigned'].index, inplace=True)

# Combining the neighborhoods that exists in one postal code
toronto_df = toronto_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()

# Changing ''Not assigned' Neighborhood to its Borough's name
toronto_df.loc[85,'Neighborhood'] = 'Queen\'s Park'

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
toronto_df.shape

(103, 3)

### Using Geo data for latitude and longitude of neighborhoods of Toronto

In [5]:
# Creating a dataframe of the latitude and longitude of the Toronto Neighborhoods

lat_long = pd.read_csv("http://cocl.us/Geospatial_data")
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
lat_long.shape

(103, 3)

In [7]:
lat_long.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [8]:
# Joining the lat_long dataframe to Neighborhoods dataframe

toronto_df.set_index("Postal Code")
lat_long.set_index("Postal Code")
neighbor=pd.merge(toronto_df, lat_long)
neighbor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    ) )

Toronto has 11 boroughs and 103 neighborhoods.


In [10]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Canada are: {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Canada are: 43.653963, -79.387207.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


### Map of Toronto with neigborhoods using Folium map

In [11]:
# Creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
northyork_data = neighbor[neighbor['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493


In [13]:
address = 'North York, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York, Toronto, CA are: {}, {}.'.format(latitude, longitude))

The geograpical coordinates of North York, Toronto, CA are: 43.7543263, -79.44911696639593.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


### Map of North York Borough of Toronto using Folium map

In [14]:
# Creating map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

### Defining credentials for Foursquare API

In [15]:
CLIENT_ID = '0X5IKAAZ5Z1SPYTTWZVZDEZ0BBKO3KCMBHVZCYUCOK2ZQXQ0'
CLIENT_SECRET = 'GCRO55XISQOUWV4T2B5WVOIQKMQT53TOFNPL2CMVO01S1IDH'
VERSION = '20180605'

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials:
CLIENT_ID: 0X5IKAAZ5Z1SPYTTWZVZDEZ0BBKO3KCMBHVZCYUCOK2ZQXQ0
CLIENT_SECRET:GCRO55XISQOUWV4T2B5WVOIQKMQT53TOFNPL2CMVO01S1IDH


### Exploring neighborhoods of North York Borough of Toronto

In [16]:
limit=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
            CLIENT_SECRET, VERSION, lat, lng, radius, limit)
            
        # making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # returning only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [17]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude'] 
                                  )

Found 244 venues in 24 neighborhoods.


In [18]:
northyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [19]:
northyork_venues.shape

(244, 7)

In [20]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",26,26,26,26,26,26
"CFB Toronto,Downsview East",3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,2,2,2,2,2,2
Downsview Northwest,4,4,4,4,4,4
Downsview West,5,5,5,5,5,5
"Downsview,North Park,Upwood Park",4,4,4,4,4,4


In [21]:
print('There are {} distinct venues in {} categories.'.format(
    len(northyork_venues['Venue Name'].unique()),len(northyork_venues['Venue Category'].unique())))

There are 204 distinct venues in 110 categories.


### Analysing each neighborhood of North York Borough of Toronto

In [22]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# moving neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview,Henry Farm,Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [23]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.052632,0.052632,0.000000,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.052632,0.0,0.052632,0.000000,0.00,0.00,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.0,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.038462,0.000000,0.00,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.076923,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.038462,0.000000,0.000000,0.00,0.0,0.

In [24]:
num_top_venues = 10

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1                Pizza Place  0.05
2              Shopping Mall  0.05
3                Gas Station  0.05
4         Frozen Yogurt Shop  0.05
5        Fried Chicken Joint  0.05
6  Middle Eastern Restaurant  0.05
7                      Diner  0.05
8                       Park  0.05
9                   Pharmacy  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00
5       Massage Studio  0.00
6             Pharmacy  0.00
7            Pet Store  0.00
8                 Park  0.00
9        Movie Theater  0.00


----Bedford Park,Lawrence Manor East----
                     venue  freq
0              Pizza Place  0.12
1       Italian Restaurant  0.08
2              Coffee Shop  0.08
3     Fast Food Restaurant  0.08
4       

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Sorting neighborhood venues of North York Borough of Toronto

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Supermarket,Middle Eastern Restaurant,Park,Deli / Bodega,Pharmacy,Pizza Place,Diner,Restaurant,Ice Cream Shop
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
2,"Bedford Park,Lawrence Manor East",Pizza Place,Sandwich Place,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Indian Restaurant,Comfort Food Restaurant,Liquor Store,Pharmacy,Juice Bar
3,"CFB Toronto,Downsview East",Park,Snack Place,Airport,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
4,Don Mills North,Baseball Field,Gym / Fitness Center,Café,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Women's Store,Electronics Store,Dog Run,Discount Store
5,Downsview Central,Food Truck,Baseball Field,Women's Store,Event Space,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
6,Downsview Northwest,Grocery Store,Gym / Fitness Center,Discount Store,Liquor Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
7,Downsview West,Grocery Store,Hotel,Shopping Mall,Bank,Park,French Restaurant,Fried Chicken Joint,Construction & Landscaping,Convenience Store,Cosmetics Shop
8,"Downsview,North Park,Upwood Park",Park,Bakery,Basketball Court,Construction & Landscaping,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
9,"Emery,Humberlea",Baseball Field,Women's Store,Event Space,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [28]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Flemingdon Park,Don Mills South
1st Most Common Venue                         Coffee Shop
2nd Most Common Venue                                 Gym
3rd Most Common Venue                    Asian Restaurant
4th Most Common Venue                          Beer Store
5th Most Common Venue                      Clothing Store
6th Most Common Venue                           Bike Shop
7th Most Common Venue                        Concert Hall
8th Most Common Venue                  Dim Sum Restaurant
9th Most Common Venue                  Chinese Restaurant
10th Most Common Venue                     Discount Store
Name: 11, dtype: object

### Clustering North York Borough of Toronto

In [30]:
# setting number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# performing k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(northyork_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 4, 0, 0, 0, 4], dtype=int32)

In [31]:
northyork_merged = northyork_data

# adding cluster labels
northyork_merged['Cluster Labels'] = kmeans.labels_

# merging northyork_grouped with northyork_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Tea Room,Convenience Store,Juice Bar,Bakery,Women's Store,Toy / Game Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,0,Cafeteria,Women's Store,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,0,Piano Bar,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


### Creating map indicating clustering labels of North York Borough of Toronto

In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining North York Borough of Toronto Clusters

In [33]:
northyork_cluster_0 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(4, northyork_merged.shape[1]))]]

northyork_cluster_1 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(4, northyork_merged.shape[1]))]]

northyork_cluster_2 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(4, northyork_merged.shape[1]))]]

northyork_cluster_3 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(4, northyork_merged.shape[1]))]]

northyork_cluster_4 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(4, northyork_merged.shape[1]))]]

In [34]:
northyork_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,-79.363452,0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
1,North York,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Tea Room,Convenience Store,Juice Bar,Bakery,Women's Store,Toy / Game Store
2,North York,-79.385975,0,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
3,North York,-79.374714,0,Cafeteria,Women's Store,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
4,North York,-79.408493,0,Piano Bar,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
6,North York,-79.400049,0,Park,Convenience Store,Bank,Flower Shop,Women's Store,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
7,North York,-79.442259,0,Pharmacy,Coffee Shop,Pizza Place,Home Service,Discount Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
8,North York,-79.329656,0,Park,Food & Drink Shop,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
10,North York,-79.340923,0,Coffee Shop,Gym,Asian Restaurant,Beer Store,Clothing Store,Bike Shop,Concert Hall,Dim Sum Restaurant,Chinese Restaurant,Discount Store
11,North York,-79.442259,0,Coffee Shop,Supermarket,Middle Eastern Restaurant,Park,Deli / Bodega,Pharmacy,Pizza Place,Diner,Restaurant,Ice Cream Shop


In [35]:
northyork_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,-79.506944,1,Grocery Store,Hotel,Shopping Mall,Bank,Park,French Restaurant,Fried Chicken Joint,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [36]:
northyork_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,-79.520999,2,Grocery Store,Gym / Fitness Center,Discount Store,Liquor Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [37]:
northyork_cluster_3

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,-79.464763,3,Clothing Store,Accessories Store,Boutique,Coffee Shop,Carpet Store,Event Space,Miscellaneous Shop,Vietnamese Restaurant,Furniture / Home Store,French Restaurant


In [38]:
northyork_cluster_4

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,-79.408493,4,Ramen Restaurant,Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Hotel,Ice Cream Shop,Plaza
9,North York,-79.352188,4,Baseball Field,Gym / Fitness Center,Café,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Women's Store,Electronics Store,Dog Run,Discount Store


### 2. Segmenting, Exploring and Clustering the neighhborhoods of Newyork City

In [39]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [40]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [43]:
neighborhood_data = newyork_data['features']
neighborhood_data

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

### Transforming the Newyork city neighborhoods data

In [44]:
# defining the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiating the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [45]:
for data in neighborhood_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [46]:
neighborhoods.shape

(306, 4)

In [47]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [48]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of New York City are: {}, {}.'.format(latitude, longitude))

The geograpical coordinates of New York City are: 40.7127281, -74.0060152.


### Map of Newyork city with neigborhoods using Folium map

In [49]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [50]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [51]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Manhattan, NY are: {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Manhattan, NY are: 40.7896239, -73.9598939.


### Map of Manhattan Borough of Newyork city with neigborhoods using Folium map

In [52]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [53]:
manhattan_data.loc[10, 'Neighborhood']

'Lenox Hill'

In [54]:
neighborhood_latitude = manhattan_data.loc[10, 'Latitude']
neighborhood_longitude = manhattan_data.loc[10, 'Longitude']

neighborhood_name = manhattan_data.loc[10, 'Neighborhood']

print('Latitude and longitude values of {} are: {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lenox Hill are: 40.76811265828733, -73.9588596881376.


### Exploring neighborhoods of Manhattan Borough of Newyork city

In [55]:
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 
    VERSION, neighborhood_latitude, neighborhood_longitude, radius, limit)

# Sending the GET request
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filtering columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filtering the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# cleaning columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Up Thai,Thai Restaurant,40.769898,-73.957598
1,sweetgreen,Salad Place,40.767128,-73.956846
2,Whiskey & Wine Off 69,Liquor Store,40.767272,-73.959544
3,Cigar Inn,Smoke Shop,40.768776,-73.956222
4,La Esquina,Taco Place,40.769451,-73.957782


In [56]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 
            VERSION, lat, lng, radius, limit)
            
        # making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # returning only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [58]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'])

Found 3305 venues in 40 neighborhoods.


In [59]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Blink Fitness,40.877271,-73.905595,Gym


In [60]:
manhattan_venues.shape

(3305, 7)

In [61]:
manhattan_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
3300,Hudson Yards,40.756658,-74.000111,George's,40.757760,-74.000963,Burger Joint
3301,Hudson Yards,40.756658,-74.000111,Big George's Smokehouse,40.757805,-74.001660,BBQ Joint
3302,Hudson Yards,40.756658,-74.000111,NYPD Mounted Unit,40.759155,-74.004121,Stables
3303,Hudson Yards,40.756658,-74.000111,Gray Line New York Sightseeing Cruises - Pier 78,40.759721,-74.003982,Harbor / Marina
3304,Hudson Yards,40.756658,-74.000111,Twilight Cruise By Citysightseeing,40.759744,-74.004096,Boat or Ferry


In [62]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,95,95,95,95,95,95
Carnegie Hill,99,99,99,99,99,99
Central Harlem,47,47,47,47,47,47
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [63]:
print('There are {} distinct venues in {} categories.'.format(
    len(manhattan_venues['Venue Name'].unique()),len(manhattan_venues['Venue Category'].unique())))

There are 2857 distinct venues in 336 categories.


### Analysing each neighborhood of Manhattan Borough of Newyork city

In [64]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

neighbor = manhattan_onehot['Neighborhood']
manhattan_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
manhattan_onehot.insert(0, 'Neighborhood', neighbor)

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Sk

In [65]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Sk

In [66]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.07
1          Hotel  0.05
2    Coffee Shop  0.05
3            Gym  0.04
4  Women's Store  0.03


----Carnegie Hill----
                 venue  freq
0          Coffee Shop  0.07
1          Pizza Place  0.05
2       Cosmetics Shop  0.04
3  Japanese Restaurant  0.03
4               Bakery  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.06
1  Fried Chicken Joint  0.04
2  American Restaurant  0.04
3                  Bar  0.04
4   Seafood Restaurant  0.04


----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1              Bakery  0.05
2  Italian Restaurant  0.04
3      Ice Cream Shop  0.04
4               Hotel  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.10
1         Cocktail Bar  0.04
2  American Restaurant  0.04
3   Salon / Barbershop  0.03
4         Optical Shop  0.03


----Civic Center----
                  venue  freq
0  Gym

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Sorting neighborhood venues of Manhattan Borough of Newyork city

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Wine Shop,Memorial Site,Women's Store,Gourmet Shop,BBQ Joint,Boat or Ferry
1,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Yoga Studio,Grocery Store,Bookstore,Café,Bakery,French Restaurant,Gym
2,Central Harlem,African Restaurant,Chinese Restaurant,Fried Chicken Joint,Bar,Cosmetics Shop,Seafood Restaurant,American Restaurant,French Restaurant,Bookstore,Caribbean Restaurant
3,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Ice Cream Shop,Theater,American Restaurant,Hotel,Nightclub,Sushi Restaurant,Tapas Restaurant
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Salon / Barbershop,Bakery,Spa,Optical Shop,Vietnamese Restaurant,Hotpot Restaurant,Asian Restaurant
5,Civic Center,Gym / Fitness Center,Italian Restaurant,French Restaurant,Hotel,Sandwich Place,Coffee Shop,Yoga Studio,Spa,Park,Cocktail Bar
6,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,Spa,American Restaurant,Wine Shop,Hotel,Sandwich Place,Mediterranean Restaurant
7,East Harlem,Mexican Restaurant,Deli / Bodega,Latin American Restaurant,Bakery,Thai Restaurant,Pizza Place,Spanish Restaurant,Café,French Restaurant,Steakhouse
8,East Village,Bar,Wine Bar,Ice Cream Shop,Chinese Restaurant,Mexican Restaurant,Pizza Place,Cocktail Bar,Vegetarian / Vegan Restaurant,Italian Restaurant,Ramen Restaurant
9,Financial District,Coffee Shop,American Restaurant,Bar,Gym,Hotel,Food Truck,Pizza Place,Italian Restaurant,Wine Shop,Gym / Fitness Center


In [69]:
neighborhoods_venues_sorted.iloc[1,]

Neighborhood                  Carnegie Hill
1st Most Common Venue           Coffee Shop
2nd Most Common Venue           Pizza Place
3rd Most Common Venue        Cosmetics Shop
4th Most Common Venue           Yoga Studio
5th Most Common Venue         Grocery Store
6th Most Common Venue             Bookstore
7th Most Common Venue                  Café
8th Most Common Venue                Bakery
9th Most Common Venue     French Restaurant
10th Most Common Venue                  Gym
Name: 1, dtype: object

### Clustering Manhattan Borough of Newyork city

In [70]:
# setting number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# performing k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(manhattan_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 4, 1, 0], dtype=int32)

In [71]:
manhattan_merged = manhattan_data

# adding clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merging manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Sandwich Place,Yoga Studio,Pharmacy,Tennis Stadium,Seafood Restaurant,Gym,Supplement Shop,Bank,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Cocktail Bar,American Restaurant,Salon / Barbershop,Bakery,Spa,Optical Shop,Vietnamese Restaurant,Hotpot Restaurant,Asian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Grocery Store,Bakery,Mobile Phone Shop,Sandwich Place,Coffee Shop,Tapas Restaurant,Gym,Latin American Restaurant,New American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Café,Lounge,Restaurant,Park,Frozen Yogurt Shop,Bakery,Caribbean Restaurant,Chinese Restaurant,Pizza Place
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Bakery,Park,Sandwich Place,Cocktail Bar,School


### Creating map indicating clustering labels of Manhattan Borough of Nework city

In [72]:
map_clusters1 = folium.Map(location=[latitude, longitude], zoom_start=11)

# setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1

### Examining Manhattan Borough of Newyork city Clusters

In [73]:
manhattan_cluster_0 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_2 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_3 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_4 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

In [74]:
manhattan_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,0,Coffee Shop,Sandwich Place,Yoga Studio,Pharmacy,Tennis Stadium,Seafood Restaurant,Gym,Supplement Shop,Bank,Donut Shop
1,Chinatown,0,Chinese Restaurant,Cocktail Bar,American Restaurant,Salon / Barbershop,Bakery,Spa,Optical Shop,Vietnamese Restaurant,Hotpot Restaurant,Asian Restaurant
2,Washington Heights,0,Café,Grocery Store,Bakery,Mobile Phone Shop,Sandwich Place,Coffee Shop,Tapas Restaurant,Gym,Latin American Restaurant,New American Restaurant
3,Inwood,0,Mexican Restaurant,Café,Lounge,Restaurant,Park,Frozen Yogurt Shop,Bakery,Caribbean Restaurant,Chinese Restaurant,Pizza Place
5,Manhattanville,0,Coffee Shop,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Seafood Restaurant,Park,Japanese Curry Restaurant,Boutique,Supermarket
6,Central Harlem,0,African Restaurant,Chinese Restaurant,Fried Chicken Joint,Bar,Cosmetics Shop,Seafood Restaurant,American Restaurant,French Restaurant,Bookstore,Caribbean Restaurant
9,Yorkville,0,Italian Restaurant,Bar,Coffee Shop,Gym,Deli / Bodega,Pizza Place,Sushi Restaurant,Japanese Restaurant,Diner,Wine Shop
14,Clinton,0,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,Spa,American Restaurant,Wine Shop,Hotel,Sandwich Place,Mediterranean Restaurant
17,Chelsea,0,Coffee Shop,Bakery,Italian Restaurant,Ice Cream Shop,Theater,American Restaurant,Hotel,Nightclub,Sushi Restaurant,Tapas Restaurant
22,Little Italy,0,Bakery,Café,Italian Restaurant,Sandwich Place,Hotpot Restaurant,Salon / Barbershop,Pizza Place,Mediterranean Restaurant,Bubble Tea Shop,Cocktail Bar


In [75]:
manhattan_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,1,Exhibit,Italian Restaurant,Bakery,Coffee Shop,Art Gallery,Juice Bar,Gym / Fitness Center,Hotel,American Restaurant,French Restaurant
11,Roosevelt Island,1,Sandwich Place,Plaza,Park,Café,Farmers Market,Soccer Field,Supermarket,Metro Station,Outdoors & Recreation,Dry Cleaner
12,Upper West Side,1,Italian Restaurant,Bar,Wine Bar,Coffee Shop,Indian Restaurant,Bakery,Café,Mediterranean Restaurant,Gym / Fitness Center,Ice Cream Shop
16,Murray Hill,1,Sandwich Place,Coffee Shop,Japanese Restaurant,American Restaurant,Hotel,Chinese Restaurant,Italian Restaurant,Gym / Fitness Center,Restaurant,Burger Joint
20,Lower East Side,1,Art Gallery,Café,Chinese Restaurant,Pizza Place,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Bakery,Cocktail Bar,Theater
36,Tudor City,1,Park,Café,Pizza Place,Mexican Restaurant,Deli / Bodega,Coffee Shop,Diner,Greek Restaurant,Dog Run,Thai Restaurant
39,Hudson Yards,1,American Restaurant,Italian Restaurant,Hotel,Gym / Fitness Center,Coffee Shop,Café,Spanish Restaurant,Park,Thai Restaurant,Bar


In [76]:
manhattan_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,2,Coffee Shop,Pizza Place,Cosmetics Shop,Yoga Studio,Grocery Store,Bookstore,Café,Bakery,French Restaurant,Gym


In [77]:
manhattan_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,3,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Bakery,Park,Sandwich Place,Cocktail Bar,School
10,Lenox Hill,3,Coffee Shop,Italian Restaurant,Pizza Place,Cocktail Bar,Sporting Goods Shop,Café,Gym,Gym / Fitness Center,Sushi Restaurant,Burger Joint
18,Greenwich Village,3,Italian Restaurant,Clothing Store,Café,Sushi Restaurant,Seafood Restaurant,French Restaurant,Indian Restaurant,Chinese Restaurant,Sandwich Place,Cocktail Bar
19,East Village,3,Bar,Wine Bar,Ice Cream Shop,Chinese Restaurant,Mexican Restaurant,Pizza Place,Cocktail Bar,Vegetarian / Vegan Restaurant,Italian Restaurant,Ramen Restaurant
29,Financial District,3,Coffee Shop,American Restaurant,Bar,Gym,Hotel,Food Truck,Pizza Place,Italian Restaurant,Wine Shop,Gym / Fitness Center


In [78]:
manhattan_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,4,Mexican Restaurant,Deli / Bodega,Latin American Restaurant,Bakery,Thai Restaurant,Pizza Place,Spanish Restaurant,Café,French Restaurant,Steakhouse
13,Lincoln Square,4,Theater,Italian Restaurant,Café,Concert Hall,Plaza,Performing Arts Venue,French Restaurant,Gym / Fitness Center,American Restaurant,Indie Movie Theater
15,Midtown,4,Hotel,Clothing Store,Sporting Goods Shop,Coffee Shop,Theater,Steakhouse,Bakery,American Restaurant,Japanese Restaurant,Bookstore
21,Tribeca,4,Italian Restaurant,American Restaurant,Park,Café,Spa,Greek Restaurant,Men's Store,Coffee Shop,Wine Bar,Wine Shop
33,Midtown South,4,Korean Restaurant,Hotel Bar,Japanese Restaurant,Hotel,Dessert Shop,Coffee Shop,American Restaurant,Boutique,Salad Place,Fried Chicken Joint
37,Stuyvesant Town,4,Boat or Ferry,Park,Bar,Baseball Field,Heliport,Gas Station,Farmers Market,German Restaurant,Gym / Fitness Center,Harbor / Marina
